# BE 521: Final Project Algorithm
Spring 2022

Desmond Young and Arjun Kanthawar

Instructions:

i. Upload our final project zipped file(final_project.zip) to google colab. 

ii. Press the run button. The notebook should do the rest. 

iii. The code outputs predictions.mat, which contains a variable predicted_dg, 
which is our predictions on the hidden test set. 


We are using a ridge regression model (alpha = 10000) that has been trained on the raw training data. 

# 1. Load Packages and Data

In [2]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig
import scipy.io
import zipfile
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [35]:

with zipfile.ZipFile('/content/final_project.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [36]:
with open('final_project/Ridge_patient1.pkl', 'rb') as f:
    ridge_pat1 = pickle.load(f)

with open('final_project/Ridge_patient2.pkl', 'rb') as f:
    ridge_pat2 = pickle.load(f)

with open('final_project/Ridge_patient3.pkl', 'rb') as f:
    ridge_pat3 = pickle.load(f)

with open('final_project/R1_raw.pkl', 'rb') as f:
    R1_raw = pickle.load(f)

with open('final_project/R2_raw.pkl', 'rb') as f:
    R2_raw = pickle.load(f)

with open('final_project/R3_raw.pkl', 'rb') as f:
    R3_raw = pickle.load(f)

In [ ]:
# Leaderboard Data
dataLeader = scipy.io.loadmat('truetest_data.mat')
patient1 = np.delete(dataLeader['truetest_data'][0][0],54,axis=1)
patient2 = np.delete(dataLeader['truetest_data'][1][0],[20,37],axis=1)
patient3 = dataLeader['truetest_data'][2][0]


# 2. Data Pre-Processing and Filtering

- Pre-process data with Common Average Reference (CAR)
- Filter Data

In [37]:
def CAR(patient):
  for i in range(len(patient)):
    
    commonAverage = np.mean(patient[i,:])
    patient[i,:] = [j-commonAverage for j in patient[i,:]]

  return patient


In [38]:
patient1_CAR = CAR(patient1)
patient2_CAR = CAR(patient2)
patient3_CAR = CAR(patient3)

In [39]:
def SigFilter(raw,fs):
  ze = np.zeros((len(raw[:,0]),len(raw[0,:])))
  fin = np.zeros((len(raw[:,0]),len(raw[0,:])))
  b,a = sig.butter(5,(1,200),btype='bandpass',fs=fs)
  b1, a1 = sig.iirnotch(60, 30, fs)
  for i in range(len(raw[0,:])):
    ze[:,i] = sig.filtfilt(b,a,raw[:,i])
    fin[:,i] = sig.filtfilt(b1,a1,ze[:,i])
  return fin

In [40]:
patient1_CAR = SigFilter(patient1_CAR,1000)
patient2_CAR = SigFilter(patient2_CAR,1000)
patient3_CAR = SigFilter(patient3_CAR,1000)

# 3. Define Features
9 features used:
- Line Length
- Area
- Average Voltage
- Variance
- Bandpower (5-15 Hz)
- Bandpower (20-25 Hz)
- Bandpower (75-115 Hz)
- Bandpower (125-160 Hz)
- Bandpower (160-175 Hz)

get_features function calculates features in a given time window





In [41]:
def NumWins(x,fs,winLen,winDisp):
  siglength = len(x) / fs
  winNums, _ = divmod((siglength - winLen), winDisp)
  return winNums + 2

In [42]:
def LL(x):
  return np.sum(np.absolute(np.ediff1d(x)))

In [43]:
def Area(x):
  return np.sum(np.abs(x))

In [44]:
def Voltage(x):
  return np.mean(x)

In [45]:
def Var(x):
  return np.var(x)

In [46]:
def Amp_5_15(x,fs):
  b,a = sig.butter(5,(5,15),btype='bandpass',fs=fs)
  signal1 = sig.filtfilt(b,a,x)
  yy = scipy.fft.fft(signal1)
  return np.sum(np.abs(yy))

In [47]:
def Amp_20_25(x,fs):
  b,a = sig.butter(5,(20,25),btype='bandpass',fs=fs)
  signal1 = sig.filtfilt(b,a,x)
  yy = scipy.fft.fft(signal1)
  return np.sum(np.abs(yy))

In [48]:
def Amp_75_115(x,fs):
  b,a = sig.butter(5,(75,115),btype='bandpass',fs=fs)
  signal1 = sig.filtfilt(b,a,x)
  yy = scipy.fft.fft(signal1)
  return np.sum(np.abs(yy))

In [49]:
def Amp_125_160(x,fs):
  b,a = sig.butter(5,(125,160),btype='bandpass',fs=fs)
  signal1 = sig.filtfilt(b,a,x)
  yy = scipy.fft.fft(signal1)
  return np.sum(np.abs(yy))

In [50]:
def Amp_160_175(x,fs):
  b,a = sig.butter(5,(160,175),btype='bandpass',fs=fs)
  signal1 = sig.filtfilt(b,a,x)
  yy = scipy.fft.fft(signal1)
  return np.sum(np.abs(yy))

In [51]:
def get_features(filtered_window, fs=1000):
  """
    Calculates features for a given filtered window. 

    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """
  window_mat = np.zeros((1,len(filtered_window[0,:])*9))
  for i in range(len(filtered_window[0,:])):
    window = filtered_window[:,i]

    window_mat[0,i*6] = LL(window)
    window_mat[0,i*6+1] = Area(window)
    window_mat[0,i*6 + 2] = Amp_75_115(window,fs)
    window_mat[0,i*6 + 3] = Amp_125_160(window,fs)
    window_mat[0,i*6 + 4] = Amp_160_175(window,fs)
    window_mat[0,i*6+5] = Voltage(window)
    window_mat[0,i*6+6] = Amp_5_15(window,fs)
    window_mat[0,i*6 + 7] = Amp_20_25(window,fs)
    window_mat[0,i*6 + 8] = Var(window)

    
    #window_mat[0,i*7 + 5] = LMP(window)
    #window_mat[0,i*7 + 6] = Var(window)
  return window_mat

# 4. Compute Features

In [52]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
  """
    Processes data through the steps of filtering and
    feature calculation and returns features. 
    
    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """
  windows = NumWins(raw_ecog,fs,window_length,window_overlap)
  window_features = np.zeros((int(windows),len(raw_ecog[0,:]) * 9))
  for i in range(0,int(windows)):
    front = int(i * window_overlap * fs)
    end = int(((i * window_overlap) + window_length) * fs)
    win = raw_ecog[front:end,:]
    feats = get_features(win, fs = fs)
    window_features[i,:] = feats
  return window_features

In [53]:


data1_CAR = get_windowed_feats(patient1_CAR,1000,0.1,0.05)
data2_CAR = get_windowed_feats(patient2_CAR,1000,0.1,0.05)
data3_CAR = get_windowed_feats(patient3_CAR,1000,0.1,0.05)



## 5. Response Matrix


In [54]:
def create_R_matrix(features, N_wind):
  """ 
  Calculates the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  
  newFeat  = np.vstack([features[0:N_wind-1],features])
  samples = len(features)
  R = np.zeros((samples,len(newFeat[0])*N_wind))
  gap = 0
  chan = 0

  for i in range(len(newFeat[0])*N_wind):
    ft_samp = newFeat[:,chan]
    R[:,i] = ft_samp[gap:samples+gap]
    gap = gap+1

    if gap == N_wind:
      gap = 0
      chan = chan + 1 # new channel
    
  
  R = np.hstack([np.ones((samples,1)), R])

  return R
  




In [55]:
# Leaderboard data
R1 = create_R_matrix(data1_CAR,14)
R2 = create_R_matrix(data2_CAR,14)
R3 = create_R_matrix(data3_CAR,14)


# 6. ML Training and Testing 

Here, we will use the R matrix with Ridge regression to predict finger flexions.

## Ridge Regression


In [56]:
# Pre-processing for Ridge Regression

sc1 = StandardScaler()
sc1.fit(R1_raw)
test1_transform = sc1.transform(R1)


sc2 = StandardScaler()
sc2.fit(R2_raw)
test2_transform = sc2.transform(R2)

sc3 = StandardScaler()
sc3.fit(R3_raw)
test3_transform = sc3.transform(R3)

In [57]:
# Predictions
ypred1 = ridge_pat1.predict(test1_transform)
ypred2 = ridge_pat2.predict(test2_transform)
ypred3 = ridge_pat3.predict(test3_transform)

## Post-Processing and Export Predictions

In [58]:
# Set all negative values to 0
for i in range(len(ypred1)):
  for j in range(5):
    if ypred1[i,j]<0:
      ypred1[i,j] = 0
    if ypred2[i,j]<0:
      ypred2[i,j] = 0
    if ypred3[i,j]<0:
      ypred3[i,j] = 0
   


In [59]:
subs = [ypred1,ypred2,ypred3]

In [60]:
subs_clean = np.zeros((115500,15))
for i in range(len(subs)):
  for j in range(len(subs[0][0,:])):
    #  subs_clean[:,j + i*5] = scipy.signal.resample(subs[i][:,j],115500)
     splined = scipy.interpolate.CubicSpline(np.arange(0,len(subs[i][:,j]),1),subs[i][:,j])
     x = np.arange(0,2949,2949/115500)
     subs_clean[:,j + i*5] = splined(x)

In [61]:
import numpy as np
from scipy.io import savemat
#create an example submission array
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = subs_clean[:,0:5]
predictions[1,0] = subs_clean[:,5:10]
predictions[2,0] = subs_clean[:,10:15]
#save the array using the right format
savemat('predictions.mat', {'predicted_dg':predictions})